# Evaluation of Stock Featues using Machine Learning Techniques

The 'data mining' game started when CAPM was proposed in 1960s - the first factor model of return. And then in 1992, Fama and Frend proposed the famous 3 factor-models with factors including 'market return', 'size' (SMB), and 'book-to-market ratio' (HML). The game was unstoppable and investors try to 'dig' out factors that well explain the market. 

The traditional $MSE$ regression has several drawbacks, including:
1. **Multi-collinearity**: features may interact with themselves, resulting in a lot of redundant features as well as large variance in coefficients
2. **Over-fit**: it minimizes the bias with the cost of large variance
3. **Hard to intepret** the resulting coefficients
4. **Not a clear rule-set** to select features

To adress all of these issues, we use the 3 famous regression models:
1. **Ridge regression**: add a penalty term to the MSE argument to shrink the size of some coefficient, but keep all features. The penalty effect is controled by a constant that multiplies the penalty term - $L1$.
2. **Lasso regression**: similar to Ridge, but allows coefficients to be zero, i.e. can remove some features. The penalty effect is controled by $L2$.
3. **Elastic Net regression**: combination of the above two, but instead of controlling L1 and L2, we usually control the sum $L=L1+L2$ (the total effect of penalty) and the $L1-ratio=L1/L$ (the weight of Ridge's penalty).

This project evaluates the prediction power of 44 different factors for all U.S. stocks listed on the NYSE, NASDAQ, or AMEX from 2018 to 2022. To ensure that all data is publicly available at the monthly timestamp and to prevent any lookahead bias, variables from columns 4 to 46 have been lagged by two months. Therefore, when performing a backtest, it is accurate to state, "According to these numbers, I plan to buy the stocks at the beginning of next month."

At the end, the 5 most effective factors are selected and interpreted in terms of their economical implication. 

**Data Source**: WRDS > CRSP/COMPUSTAT merged monthly (2018-2022)

In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt # python's plotting package
from sklearn.metrics import mean_squared_error as mse

In [24]:
test = pd.read_csv("data/test.csv")
train = pd.read_csv("data/train.csv")
validation = pd.read_csv("data/validation.csv")
print(f"test:{len(test)}, train:{len(train)}, validation:{len(validation)}.")

test:4359, train:14020, validation:4826.


In [25]:
print(train.iloc[len(train)-1])

permno             93436.000000
date              202012.000000
ret                    0.243252
rd_sale                0.087162
equity_invcap         -0.535651
npm                   -0.667802
ps                     5.368845
gprof                 -0.249735
ptpm                  -0.671354
aftret_eq              0.008141
opmad                 -0.747909
opmbd                 -0.696064
gpm                   -0.687397
debt_assets            0.280306
sale_invcap           -0.006851
adv_sale              -0.398256
staff_sale            -0.385907
de_ratio              -0.008646
aftret_equity          0.008149
capital_ratio          0.455894
pcf                    4.213743
accrual               -1.061972
at_turn                0.065345
lt_debt                0.115555
debt_invcap            0.335739
roa                   -0.207780
capei                 -0.354029
debt_at                0.563658
totdebt_invcap         0.556521
cash_debt             -0.506449
evm                    0.369026
cash_lt 

In [26]:
# data = pd.read_csv("F-F_Research_Data_Factors.csv")
# RF = pd.DataFrame(data=data.query("201801<= date <= 202212")[["date","RF"]], columns=["date","RF"]).reset_index(drop=True)
# RF.to_csv("US_rf_rate.csv")
# https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html#Research

In [27]:
rf = pd.read_csv("data/US_rf_rate.csv")
rf['RF_m'] = rf['RF']/12

def calculate_premium(data):
    """
    return a df with three more columns = ['RF','RF_m','premium']
    """
    data = pd.merge(test, rf, on="date", how='left').sort_values(by='date')
    data['premium'] = data['ret'] - data['RF_m']
    return data

## 1. Calculate premium return

In [28]:
test_data = calculate_premium(test)
train_data = calculate_premium(train)
valid_data = calculate_premium(validation)

## 2. Machine Learning Models

### Standardize all data 

In [29]:
from sklearn.preprocessing import StandardScaler

In [30]:
# standardized data
def standardize(data: pd.DataFrame,scaler: StandardScaler = None):
    scaled = data.copy()
    scaled.iloc[:,3:47] = scaler.transform(scaled.iloc[:,3:47])
    return scaled

# initialize StandardScaler
scaler = StandardScaler()
scaler.fit(train_data.iloc[:,3:47])

test_data_stand = standardize(test_data, scaler)
train_data_stand = standardize(train_data, scaler)
valid_data_stand = standardize(valid_data, scaler)

In [31]:
scaler.fit(train_data['premium'].values.reshape(-1, 1))

test_data_stand['premium'] = scaler.transform(test_data['premium'].values.reshape(-1, 1)).ravel()
train_data_stand['premium'] = scaler.transform(train_data['premium'].values.reshape(-1, 1)).ravel()
valid_data_stand['premium'] = scaler.transform(valid_data['premium'].values.reshape(-1, 1)).ravel()

In [32]:
print("Feature standard deviation:", np.mean(np.std(train_data_stand.iloc[:,3:47], axis=0)))
print("Target festure standard deviation:", np.std(train_data_stand['premium']))
print("Feature mean:", np.mean(np.mean(train_data_stand.iloc[:,3:47], axis=0)))
print("Target feature mean:", np.mean(train_data_stand['premium']))

print("Standardization complete")

Feature standard deviation: 0.9999999999999996
Target festure standard deviation: 0.9999999999999983
Feature mean: 8.5670716617929e-19
Target feature mean: -6.5202361620564364e-18
Standardization complete


### 2.1 Ridge

In [33]:
# Importing Ridge
from sklearn.linear_model import Ridge

In [34]:
scaler = StandardScaler()

train_data_scaled = scaler.fit_transform(train_data.iloc[:,3:47])
valid_data_scaled = scaler.transform(valid_data.iloc[:,3:47])
print(type(train_data_scaled))

<class 'numpy.ndarray'>


In [35]:
# The alpha used by Python's ridge should be the lambda in Hull's book times the number of observations, i.e.the number of observation in training set
# alphas=[0.1*num_train, 0.2*num_train,0.3*num_train,0.4*num_train,0.5*num_train,0.6*num_train,0.7*num_train,0.8*num_train,0.9*num_train,1*num_train]
num_train = len(train_data_stand)
alphas = [0.1*num_train*i for i in range(1,11)]
mses=[]
for alpha in alphas:
    ridge=Ridge(alpha=alpha)
    ridge.fit(train_data_stand.iloc[:,3:47].values,train_data_stand['premium'].values)
    pred=ridge.predict(valid_data_stand.iloc[:,3:47].values)
    mses.append(mse(valid_data_stand['premium'],pred))
    # print(mse(valid_data_stand['premium'],pred))

best_ridge_alpha = alphas[mses.index(min(mses))]/num_train

print(f"\nThe minimum mse is {min(mses)}, corresponding to lambda = {best_ridge_alpha}.")


The minimum mse is 0.9754538048762509, corresponding to lambda = 0.1.


### 2.2 Lasso

In [36]:
# Import Lasso
from sklearn.linear_model import Lasso

In [37]:
# We now consider different lambda values. The alphas are half the lambdas
from sklearn.metrics import mean_squared_error

# alphas = [0.001/2, 0.002/2,0.003/2,0.004/2,0.005/2,0.006/2,0.007/2,0.008/2,0.009/2,0.01/2]
alphas = [0.001, 0.002,0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01]
mses = []

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(train_data_stand.iloc[:,3:47].values,train_data_stand['premium'].values)
    pred = lasso.predict(valid_data_stand.iloc[:,3:47].values)
    mse_value = mean_squared_error(valid_data_stand['premium'], pred)
    mses.append(mse_value)
    # print(mse_value)

best_lasso_alpha = alphas[mses.index(min(mses))]

print(f"\nThe minimum mse is {min(mses)}, corresponding to lambda = {alphas[mses.index(min(mses))]}.")


The minimum mse is 0.9709377858509662, corresponding to lambda = 0.001.


### 2.3 Elestic Net

In [38]:
# Import Elastic Net
from sklearn.linear_model import ElasticNet

In [39]:
# alphas = [0.001/2, 0.002/2,0.003/2,0.004/2,0.005/2,0.006/2,0.007/2,0.008/2,0.009/2,0.01/2]
lambdas = [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01]
l1_ratios = [0.1, 0.5, 0.9, 0.2, 0.7]

results = []

# Train model and calculate MSE
for alpha in lambdas:
    for l1_ratio in l1_ratios:
        elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
        elastic_net.fit(train_data_stand.iloc[:,3:47].values,train_data_stand['premium'].values)
        pred = elastic_net.predict(valid_data_stand.iloc[:,3:47].values)
        error = mean_squared_error(valid_data_stand['premium'], pred)
        results.append((alpha, l1_ratio, error))
        # print(f"Alpha: {alpha}, L1 Ratio: {l1_ratio}, MSE: {error}")

# 找到最佳参数
best_result = min(results, key=lambda x: x[2])
best_en_alpha, best_en_l1_ratio, best_mse = best_result


print(f"\n=== Best Parametors ===")
print(f"Best Alpha: {best_en_alpha:.6f}")
print(f"Best L1 Ratio: {best_en_l1_ratio}")
print(f"Best MSE: {best_mse:.6f}")


=== Best Parametors ===
Best Alpha: 0.001000
Best L1 Ratio: 0.1
Best MSE: 0.970474


## 3. Choose the best model based on Test data

In [40]:
from sklearn.metrics import mean_squared_error, r2_score

# Initiate the three models with the best (least MSE) alphas
ridge_best = Ridge(alpha=best_ridge_alpha, random_state=42)
lasso_best = Lasso(alpha=best_lasso_alpha, random_state=42, max_iter=10000)
elastic_net_best = ElasticNet(alpha=best_en_alpha, l1_ratio=best_en_l1_ratio, 
                             random_state=42, max_iter=10000)

models = {
    'Ridge': ridge_best,
    'Lasso': lasso_best, 
    'Elastic Net': elastic_net_best
}

X_test = test_data_stand.iloc[:,3:47]
y_test = test_data_stand['premium'].values

# store the results
results = []

print("\n=== Individual Model Performance on Test Set ===\n")

for name, model in models.items():
    # re-train the models
    model.fit(train_data_stand.iloc[:,3:47].values, train_data_stand['premium'].values)
    
    # predict on the test data
    y_pred = model.predict(X_test)
    
    # calculate MSE and r2 score
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'MSE': mse,
        'R²': r2,
        'Parameters': f"alpha={getattr(model, 'alpha', 'N/A')}, l1_ratio={getattr(model, 'l1_ratio', 'N/A')}"
    })
    
    # print(f"{name}:")
    # print(f"  MSE: {mse:.6f}")
    # print(f"  R²: {r2:.4f}")
    # if hasattr(model, 'l1_ratio'):
        # print(f"  Parameters: alpha={model.alpha:.6f}, l1_ratio={model.l1_ratio}")
    # else:
        # print(f"  Parameters: alpha={model.alpha:.6f}")

results_df = pd.DataFrame(results)
print('\n'+results_df.to_string())
print(f"\nThe best model with lowest MSE is {min(results, key=lambda x: x['MSE'])['Model']}\n")


=== Individual Model Performance on Test Set ===



d:\Applications\Coding\Anaconda\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but Ridge was fitted without feature names
  warnings.warn(
d:\Applications\Coding\Anaconda\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(



         Model       MSE        R²                 Parameters
0        Ridge  0.970432  0.029568    alpha=0.1, l1_ratio=N/A
1        Lasso  0.970938  0.029062  alpha=0.001, l1_ratio=1.0
2  Elastic Net  0.970474  0.029526  alpha=0.001, l1_ratio=0.1

The best model with lowest MSE is Ridge



d:\Applications\Coding\Anaconda\Lib\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(


## 4. Coefficient Evaluation for the Best Model

$1 / (2 * n_{samples}) * ||y - Xw||^2_2 + alpha * l1_{ratio} * ||w||_1 + 0.5 * alpha * (1 - l1_{ratio}) * ||w||^2_2$

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("=== Best Model Coefficients Analysis ===")

best_model = elastic_net_best

# re-train to get the coeficients
best_model.fit(train_data_stand.iloc[:,3:47].values, train_data_stand['premium'].values)

# predict on the test data
X_test = test_data_stand.iloc[:,3:47].values
y_test = test_data_stand['premium'].values
y_pred = best_model.predict(X_test)

# grab coef
coefficients = best_model.coef_
feature_names = test_data_stand.columns[3:47]

# create DataFrame for coef
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Abs_Coefficient': np.abs(coefficients)
}).sort_values('Abs_Coefficient', ascending=False)

print(f"Best Model: {type(best_model).__name__}")
print(f"Number of features: {len(coef_df)}")
print(f"Non-zero coefficients: {np.sum(coefficients != 0)}")
print(f"Sparsity: {np.sum(coefficients != 0) / len(coefficients) * 100:.1f}%")

print("\nAll Coefficients (sorted by absolute value):")
print(coef_df.iloc[0:6].to_string(index=False))

=== Best Model Coefficients Analysis ===
Best Model: ElasticNet
Number of features: 44
Non-zero coefficients: 44
Sparsity: 100.0%

All Coefficients (sorted by absolute value):
      Feature  Coefficient  Abs_Coefficient
equity_invcap    -0.189142         0.189142
        opmbd     0.187162         0.187162
          evm    -0.180207         0.180207
      debt_at    -0.162934         0.162934
          roa     0.156375         0.156375
  debt_ebitda     0.146821         0.146821


In [42]:
# grab the top 5 features with highest weights
top_5_factors = coef_df.head(5).copy()
top_5_factors['Rank'] = range(1, 6)

print("\n" + "="*70)
print("TOP 5 MOST SIGNIFICANT FACTORS INFLUENCING STOCK RETURNS")
print("="*70)

for i, (idx, row) in enumerate(top_5_factors.iterrows(), 1):
    print(f"\n#{i}: {row['Feature']}")
    print(f"   Coefficient: {row['Coefficient']:.6f}")
    print(f"   Absolute Impact: {row['Abs_Coefficient']:.6f}")
    print(f"   Direction: {'Positive' if row['Coefficient'] > 0 else 'Negative'} relationship with returns")


TOP 5 MOST SIGNIFICANT FACTORS INFLUENCING STOCK RETURNS

#1: equity_invcap
   Coefficient: -0.189142
   Absolute Impact: 0.189142
   Direction: Negative relationship with returns

#2: opmbd
   Coefficient: 0.187162
   Absolute Impact: 0.187162
   Direction: Positive relationship with returns

#3: evm
   Coefficient: -0.180207
   Absolute Impact: 0.180207
   Direction: Negative relationship with returns

#4: debt_at
   Coefficient: -0.162934
   Absolute Impact: 0.162934
   Direction: Negative relationship with returns

#5: roa
   Coefficient: 0.156375
   Absolute Impact: 0.156375
   Direction: Positive relationship with returns


## 5. Economic Interpretations of the Top 5

### 1: evm: Multiple of Enterprise Value to EBITDA
Economic Interpretation: Higher EVM predicts lower returns, implying that companies with elevated EV/EBITDA ratios should be avoided. It is important to invest in companies that have reasonable prices relative to their operating earnings.
### 2: opmbd: Operating Income before Depreciation as a Fraction of Sales
Economic Interpretation: Companies with higher operating profit margins are expected to deliver superior returns since this factor reflects the company's economic moat and pricing power. It suggests seeking companies with strong and sustainable operating margins.
### 3: equity_invcap: Common Equity/Invested Capital
Economic Interpretation: Higher equity proportion in capital structure predicts lower returns. Over-reliance on equity financing may dilute returns, while reasonable debt leverage with careful risk management can increase capital efficiency. This may also indicate immature companies with less established credit capacity. Therefore, we prefer companies with balanced capital structures.
### 4: debt_ebitda: Total Debt/EBITDA
Economic Interpretation: This factor reinforces the economic interpretation of factor three from a debt perspective, suggesting that higher debt levels yield higher predicted stock returns. This could be due to efficient use of financial leverage which enhances shareholder returns. However, it is worth noting that excessive leverage should be avoided.
### 5: ps: Multiple of Market Value of Equity to Sales
Economic Interpretation: Higher price-to-sales multiples predict better performance, which seems counterintuitive to value investing. However, we should take growth expectations into consideration because the market pays a premium for return potential. Therefore, valuation naturally varies with revenue quality and growth prospects, which positively correlate with returns.
Conclusion: We believe these factors generally correspond with well-known trading strategies, such as Warren Buffett's value investment approach. Value investing considers profitability (opmbd), overvaluation (evm), and financial health (debt_ebitda and equity_invcap), which is consistent with four out of the five most important factors in our model.